In [1]:
import sys
import threading
import concurrent.futures

In [2]:
def load_graph_txt(file_name):
    num_v = 875714
    graph = {i:[] for i in range(1, num_v+1)}
    r_graph = {i:[] for i in range(1, num_v+1)}
    with open(file_name) as f:
        for line in f:
            line_split = line.split()
            head = int(line_split[0])
            tail = int(line_split[1])
            
            graph[head].append(tail)
            r_graph[tail].append(head)

    return graph, r_graph

In [3]:
graph, r_graph = load_graph_txt('SCC.txt')

In [8]:
def topo_sort(graph):
    all_vertices = graph.keys()
    explored = {v_i:False for v_i in all_vertices}
    order_stack = []
    for v_i in all_vertices:
        if explored[v_i] is False:
            DFS_topo(graph, v_i, explored, order_stack)
    return order_stack

In [9]:
def DFS_topo(graph, v, explored, order_stack):
    explored[v] = True
    for w in graph[v]:
        if explored[w] is False:
            DFS_topo(graph, w, explored, order_stack)
    order_stack.append(v)

In [10]:
def find_SCC(graph, order_stack):
    global SCC_num
    SCC_num = 0
    SCC_dict = dict()
    
    all_vertices = graph.keys()
    explored = {v_i:False for v_i in all_vertices}
    while len(order_stack) > 0:
        v = order_stack.pop()
        if explored[v] is False:
            SCC_num = SCC_num + 1
            DFS_SCC(graph, v, explored, SCC_dict)
    
    return SCC_dict

In [11]:
def DFS_SCC(graph, v, explored, SCC_dict):
    global SCC_num
    SCC_dict[v] = SCC_num
    explored[v] = True
    for w in graph[v]:
        if explored[w] is False:
            DFS_SCC(graph, w, explored, SCC_dict) 

In [12]:
def main():
    order = topo_sort(r_graph)
    scc = find_SCC(graph, order)
    return scc

In [13]:
threading.stack_size(67108864) # 64MB stack
sys.setrecursionlimit(2 ** 20)  # approx 1 million recursions

with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(main)
    scc_result = future.result()

In [14]:
scc_occur = {}
for k in scc_result.keys():
    scc_num = scc_result[k]
    if scc_num in scc_occur.keys():
        scc_occur[scc_num] = scc_occur[scc_num] + 1
    else:
        scc_occur[scc_num] = 1

In [20]:
sorted(scc_occur.values())[-5:]

[211, 313, 459, 968, 434821]

In [ ]:
testcase1 = {1:[2], 2:[3], 3:[1,4], 4:[5], 5:[6], 6:[4]}
reverse_testcase1 = {1:[3], 2:[1], 3:[2], 4:[3, 6], 5:[4], 6:[5]}